***GENERATED CODE FOR classifyingvehicle PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'VIN (1-10)', 'transformation_label': 'String Indexer'}], 'feature': 'VIN (1-10)', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '1C4JJXN61P', 'max': 'YV4H600AXP', 'missing': '0', 'distinct': '415'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'VIN (1-10)'}, {'feature_label': 'VIN (1-10)', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('VIN (1-10)')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'County', 'transformation_label': 'String Indexer'}], 'feature': 'County', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Benton', 'max': 'Yakima', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'County'}, {'feature_label': 'County', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('County')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Anacortes', 'max': 'Yelm', 'missing': '0', 'distinct': '77'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'WA', 'max': 'WA', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Make', 'transformation_label': 'String Indexer'}], 'feature': 'Make', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AUDI', 'max': 'VOLVO', 'missing': '0', 'distinct': '24'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Make'}, {'feature_label': 'Make', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Make')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Model', 'transformation_label': 'String Indexer'}], 'feature': 'Model', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '500.0', 'stddev': '0.0', 'min': '330E', 'max': 'XC90', 'missing': '0', 'distinct': '67'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Model'}, {'feature_label': 'Model', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Model')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Electric Vehicle Type', 'transformation_label': 'String Indexer'}], 'feature': 'Electric Vehicle Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Battery Electric Vehicle (BEV)', 'max': 'Plug-in Hybrid Electric Vehicle (PHEV)', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Electric Vehicle Type'}, {'feature_label': 'Electric Vehicle Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Electric Vehicle Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'transformation_label': 'String Indexer'}], 'feature': 'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Clean Alternative Fuel Vehicle Eligible', 'max': 'Not eligible due to low battery range', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Clean Alternative Fuel Ve...'}, {'feature_label': 'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Clean Alternative Fuel Vehicle (CAFV) Eligibility')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Vehicle Location', 'transformation_label': 'String Indexer'}], 'feature': 'Vehicle Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'POINT (-117.0774369 47.2271563)', 'max': 'POINT (-96.951223 33.1768139)', 'missing': '0', 'distinct': '150'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Vehicle Location'}, {'feature_label': 'Vehicle Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Vehicle Location')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Electric Utility', 'transformation_label': 'String Indexer'}], 'feature': 'Electric Utility', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||INLAND POWER & LIGHT COMPANY', 'max': 'PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Electric Utility'}, {'feature_label': 'Electric Utility', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Electric Utility')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run classifyingvehicleHooks.ipynb
try:
	#sourcePreExecutionHook()

	electricvehicledata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Electric_Vehicle_Data.csv', 'filename': 'Electric_Vehicle_Data.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(electricvehicledata)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classifyingvehicleHooks.ipynb
try:
	#transformationPreExecutionHook()

	classifyingvehicleautofe = TransformationMain.run(electricvehicledata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "VIN (1-10)", "transformation_label": "String Indexer"}], "feature": "VIN (1-10)", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1C4JJXN61P", "max": "YV4H600AXP", "missing": "0", "distinct": "415"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "VIN (1-10)"}, {"transformationsData": [{"feature_label": "County", "transformation_label": "String Indexer"}], "feature": "County", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Benton", "max": "Yakima", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "County"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Anacortes", "max": "Yelm", "missing": "0", "distinct": "77"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "WA", "max": "WA", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Postal Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "98244.17", "stddev": "1450.05", "min": "75068", "max": "99362", "missing": "0"}, "updatedLabel": "Postal Code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Model Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2018.8", "stddev": "3.15", "min": "2011", "max": "2024", "missing": "0"}, "updatedLabel": "Model Year"}, {"transformationsData": [{"feature_label": "Make", "transformation_label": "String Indexer"}], "feature": "Make", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AUDI", "max": "VOLVO", "missing": "0", "distinct": "24"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Make"}, {"transformationsData": [{"feature_label": "Model", "transformation_label": "String Indexer"}], "feature": "Model", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "500.0", "stddev": "0.0", "min": "330E", "max": "XC90", "missing": "0", "distinct": "67"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Model"}, {"transformationsData": [{"feature_label": "Electric Vehicle Type", "transformation_label": "String Indexer"}], "feature": "Electric Vehicle Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Battery Electric Vehicle (BEV)", "max": "Plug-in Hybrid Electric Vehicle (PHEV)", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Electric Vehicle Type"}, {"transformationsData": [{"feature_label": "Clean Alternative Fuel Vehicle (CAFV) Eligibility", "transformation_label": "String Indexer"}], "feature": "Clean Alternative Fuel Vehicle (CAFV) Eligibility", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Clean Alternative Fuel Vehicle Eligible", "max": "Not eligible due to low battery range", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Clean Alternative Fuel Ve..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Electric Range", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "57.47", "stddev": "90.79", "min": "0", "max": "322", "missing": "0"}, "updatedLabel": "Electric Range"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Base MSRP", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2063.89", "stddev": "10873.38", "min": "0", "max": "69900", "missing": "0"}, "updatedLabel": "Base MSRP"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Legislative District", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "30.73", "stddev": "14.71", "min": "1", "max": "49", "missing": "2"}, "updatedLabel": "Legislative District"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DOL Vehicle ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "200515794.42", "stddev": "88826176.04", "min": "1815593", "max": "479254772", "missing": "0"}, "updatedLabel": "DOL Vehicle ID"}, {"transformationsData": [{"feature_label": "Vehicle Location", "transformation_label": "String Indexer"}], "feature": "Vehicle Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "POINT (-117.0774369 47.2271563)", "max": "POINT (-96.951223 33.1768139)", "missing": "0", "distinct": "150"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Vehicle Location"}, {"transformationsData": [{"feature_label": "Electric Utility", "transformation_label": "String Indexer"}], "feature": "Electric Utility", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||INLAND POWER & LIGHT COMPANY", "max": "PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Electric Utility"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "2020 Census Tract", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "53035084240.51", "stddev": "14195959.88", "min": "53005010817", "max": "53077003400", "missing": "0"}, "updatedLabel": "2020 Census Tract"}]}))

	#transformationPostExecutionHook(classifyingvehicleautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classifyingvehicleHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(classifyingvehicleautofe, ["Postal Code", "Model Year", "Electric Range", "Base MSRP", "Legislative District", "DOL Vehicle ID", "2020 Census Tract", "VIN (1-10)_stringindexer", "County_stringindexer", "City_stringindexer", "State_stringindexer", "Make_stringindexer", "Model_stringindexer", "Clean Alternative Fuel Vehicle (CAFV) Eligibility_stringindexer", "Vehicle Location_stringindexer", "Electric Utility_stringindexer"], "Electric Vehicle Type_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

